In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as f
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from sklearn import preprocessing
from tqdm import tqdm
from sklearn.preprocessing import FunctionTransformer
import seaborn as sns 
from scipy import stats
import statsmodels.api as sm

from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor, BaggingRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import ElasticNet, Lasso
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import SVR
from sklearn.model_selection import KFold, cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import RobustScaler
import h2o
from h2o.automl import H2OAutoML
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.neural_network import MLPRegressor
from scipy.stats import norm
import copy
from sklearn.model_selection import KFold
from catboost import CatBoostRegressor, Pool, metrics, cv
import xgboost as xgb
from scipy.stats import gmean
from sklearn.impute import KNNImputer
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder

In [2]:
add_train = pd.read_csv('./n_train.csv')
add_test = pd.read_csv('./n_test.csv')

In [3]:
climate_type_dict = {'1':'BSk',
 '9': 'Dfb',
 '4': 'Cfa',
 '7': 'Csb',
 '8': 'Dfa',
 '3': 'BWk',
 '10': 'Dfc',
 '2': 'BWh',
 '6': 'Csa',
 '11': 'Dsb',
 '0': 'BSh',
 '5': 'Cfb',
 '12': 'Dsc',
 '13': 'Dwa',
 '14': 'Dwb'}

In [4]:
all_cols = add_train.columns.tolist()

In [5]:
target=["contest-tmp2m-14d__tmp2m"]

In [6]:
train_df =  pd.read_csv('fe_v1_train.csv')
test_df  =  pd.read_csv('fe_v1_test.csv')
for c in train_df.columns:
    if train_df[c].dtypes != 'float64':
        print(c)

climateregions__climateregion


In [7]:
exclude_cols = ['index', 'startdate']
best_cols = [c for c in train_df.columns if c not in exclude_cols and c not in target]
print("Main features:", len(best_cols))

Main features: 63


In [8]:
for c in all_cols:
    if 'contest-precip-14d' in c:
        print(c)

contest-precip-14d__precip


In [9]:
def calculate_season(month):
    if month == 12 or month == 1 or month == 2:
        return 4
    if month == 9 or month == 10 or month == 11:
        return 3
    if month == 7 or month == 8 or month == 6:
        return 2
    if month == 3 or month == 4 or month == 5:
        return 1

In [10]:
train_df['month'] = add_train['month']
test_df['month'] = add_test['month']

train_df['season'] = train_df['month'].apply(calculate_season)
test_df['season'] = test_df['month'].apply(calculate_season)

train_df['loc_group'] = add_train['loc_group']
test_df['loc_group'] = add_test['loc_group']

train_df['year'] = add_train['year']
test_df['year'] = add_test['year']

def map_climate_type(x):
    return climate_type_dict[str(x)]

train_df['climate_type'] = train_df['climateregions__climateregion'].apply(map_climate_type)
test_df['climate_type'] = train_df['climateregions__climateregion'].apply(map_climate_type)

precip_col = 'contest-precip-14d__precip'
wind_col = 'contest-wind-h500-14d__wind-hgt-500'

group_cols = ['loc_group', 'climate_type', 'year', 'season', 'month']
season_group_cols = ['loc_group', 'climate_type', 'year', 'season']


In [11]:
slp_col = 'contest-slp-14d__slp'
ccsm3_col = 'nmme-tmp2m-34w__ccsm3'
pres_col = 'contest-pres-sfc-gauss-14d__pres'
precip_col = 'contest-precip-14d__precip'
stt_col = 'sst-2010-1'

In [12]:
print(train_df[pres_col].min(), train_df[pres_col].max())
print(test_df[pres_col].min(), test_df[pres_col].max())

74839.94 102845.57
75925.02 102119.93


In [13]:
print(train_df[stt_col].min(), train_df[stt_col].max())
print(test_df[stt_col].min(), test_df[stt_col].max())

-219.87 352.2
89.73 345.33


In [14]:
print(train_df[precip_col].min(), train_df[precip_col].max())
print(test_df[precip_col].min(), test_df[precip_col].max())

0.0 502.63
0.0 439.54


In [15]:
# train_df[pres_col] = train_df[pres_col] - 100000
# test_df[pres_col] = test_df[pres_col] - 100000

In [16]:
print(train_df[slp_col].min(), train_df[slp_col].max())
print(test_df[slp_col].min(), test_df[slp_col].max())

100379.21 103275.84
100593.22 102258.27


In [17]:
print(train_df[wind_col].min(), train_df[wind_col].max())
print(test_df[wind_col].min(), test_df[wind_col].max())

5289.25 5958.2
5504.59 5925.56


In [18]:
train_df[wind_col] = train_df[wind_col] - 5000
test_df[wind_col] = test_df[wind_col] - 5000

In [19]:
train_df[slp_col] = train_df[slp_col] - 100000
test_df[slp_col] = test_df[slp_col] - 100000

In [20]:
check_cols = [c for c in train_df.columns if c not in target and c not in exclude_cols]

df = pd.concat([train_df[check_cols], test_df[check_cols]], axis=0)
df.shape

(407088, 68)

In [21]:
def create_df_by_group_and_agg(dataframe, grouped_cols, action, my_col, return_name):
    if action == "first":
        ans_df = dataframe.groupby(grouped_cols)[my_col].first().reset_index()
    elif action == "min":
        ans_df = dataframe.groupby(grouped_cols)[my_col].min().reset_index()
    elif action == "max":
        ans_df = dataframe.groupby(grouped_cols)[my_col].max().reset_index()
    elif action == "mean":
        ans_df = dataframe.groupby(grouped_cols)[my_col].mean().reset_index()
    print("Nan")
    ans_df = ans_df.rename(columns={my_col: return_name})
    for c in ans_df.columns:
        if ans_df[c].isnull().sum() > 0:
            print(c, ans_df[c].isnull().sum())
    return ans_df

In [22]:
def assign_train_test_df(my_col):
    train_df[my_col] = df[:len(train_df)][my_col].tolist()
    test_df[my_col] = df[len(train_df):][my_col].tolist()

In [23]:
first_slp_df = create_df_by_group_and_agg(df, season_group_cols, "first", slp_col, "first_slp")
first_slp_df.head(3)

Nan


,loc_group,climate_type,year,season,first_slp
0,0,BSh,2014,3,1352.08
1,0,BSh,2014,4,2072.81
2,0,BSh,2015,1,1917.24


In [24]:
first_wind_df = create_df_by_group_and_agg(df, season_group_cols, "first", wind_col, "first_wind")
first_wind_df.head(3)

Nan


,loc_group,climate_type,year,season,first_wind
0,0,BSh,2014,3,899.66
1,0,BSh,2014,4,847.35
2,0,BSh,2015,1,804.47


In [25]:
first_precip_df = create_df_by_group_and_agg(df, season_group_cols, "first", precip_col, "first_precip")
first_precip_df.head(3)

Nan


,loc_group,climate_type,year,season,first_precip
0,0,BSh,2014,3,94.31
1,0,BSh,2014,4,25.53
2,0,BSh,2015,1,31.94


In [26]:
first_ccsm3_df = create_df_by_group_and_agg(df, season_group_cols, "first", ccsm3_col, "first_ccsm3")
first_ccsm3_df.head(3)

Nan


,loc_group,climate_type,year,season,first_ccsm3
0,0,BSh,2014,3,27.57
1,0,BSh,2014,4,6.66
2,0,BSh,2015,1,15.25


In [27]:
first_pres_df = create_df_by_group_and_agg(df, season_group_cols, "first", ccsm3_col, "first_pres")
first_pres_df.head(3)

Nan


,loc_group,climate_type,year,season,first_pres
0,0,BSh,2014,3,27.57
1,0,BSh,2014,4,6.66
2,0,BSh,2015,1,15.25


In [28]:
first_sst_df = create_df_by_group_and_agg(df, season_group_cols, "first", stt_col, "first_sst")
first_sst_df.head(3)

Nan


,loc_group,climate_type,year,season,first_sst
0,0,BSh,2014,3,352.20
1,0,BSh,2014,4,-27.73
2,0,BSh,2015,1,-182.10


In [29]:
first_slp_df_month = create_df_by_group_and_agg(df, group_cols, "first", slp_col, "first_slp_month")
first_slp_df_month.head(3)

Nan


,loc_group,climate_type,year,season,month,first_slp_month
0,0,BSh,2014,3,9,1352.08
1,0,BSh,2014,3,10,1254.96
2,0,BSh,2014,3,11,1952.19


In [30]:
df = df.merge(first_slp_df, how='left', on=season_group_cols)
df['diff_slp_first'] = df[slp_col] - df['first_slp']
assign_train_test_df('diff_slp_first')

In [31]:
df = df.merge(first_precip_df, how='left', on=season_group_cols)
df['diff_precip_first'] = df[precip_col] - df['first_precip']
assign_train_test_df('diff_precip_first')

In [32]:
df = df.merge(first_sst_df, how='left', on=season_group_cols)
df['diff_sst_first'] = df[stt_col] - df['first_sst']
assign_train_test_df('diff_sst_first')

In [33]:
df = df.merge(first_pres_df, how='left', on=season_group_cols)
df['diff_pres_first'] = df[pres_col] - df['first_pres']
assign_train_test_df('diff_pres_first')

In [34]:
df = df.merge(first_wind_df, how='left', on=season_group_cols)
df['diff_wind_first'] = df[wind_col] - df['first_wind']
assign_train_test_df('diff_wind_first')

In [35]:
df = df.merge(first_ccsm3_df, how='left', on=season_group_cols)
df['diff_ccsm3_first'] = df[ccsm3_col] - df['first_ccsm3']
assign_train_test_df('diff_ccsm3_first')

In [36]:
df[f'{precip_col}_bins'] = pd.qcut(df[precip_col], q=5, labels=[f'{precip_col}_{i}' for i in range(5)])
le = LabelEncoder().fit(np.unique(df[f'{precip_col}_bins'].unique().tolist()))
df[f'{precip_col}_bins'] = le.transform(df[f'{precip_col}_bins']) + 1
df[f'{precip_col}_bins'] = df[f'{precip_col}_bins'].replace(np.nan, 0).astype('int').astype('category')
assign_train_test_df(f'{precip_col}_bins')

In [41]:
features = best_cols + [
    'diff_slp_first',
    'diff_wind_first',
    'diff_sst_first',
    'diff_precip_first'
]

In [43]:
best_cols

['contest-wind-h500-14d__wind-hgt-500',
 'contest-slp-14d__slp',
 'nmme-tmp2m-34w__ccsm3',
 'elevation__elevation',
 'lon',
 'contest-prwtr-eatm-14d__prwtr',
 'lat',
 'climateregions__climateregion',
 'contest-pres-sfc-gauss-14d__pres',
 'season_sin',
 'day_of_year_sin',
 'contest-precip-14d__precip',
 'contest-wind-uwnd-250-14d__wind-uwnd-250',
 'nmme-prate-34w__cfsv2',
 'nmme-prate-34w__nasa',
 'nmme-prate-56w__gfdlflora',
 'wind-uwnd-250-2010-7',
 'contest-wind-vwnd-925-14d__wind-vwnd-925',
 'nmme-prate-34w__nmmemean',
 'nmme0-prate-34w__ccsm30',
 'contest-wind-h850-14d__wind-hgt-850',
 'contest-wind-uwnd-925-14d__wind-uwnd-925',
 'nmme0-prate-56w__cfsv20',
 'nmme-prate-34w__cancm3',
 'contest-rhum-sig995-14d__rhum',
 'nmme-prate-34w__gfdlflorb',
 'wind-hgt-850-2010-4',
 'contest-wind-vwnd-250-14d__wind-vwnd-250',
 'wind-hgt-100-2010-2',
 'wind-uwnd-250-2010-18',
 'wind-hgt-10-2010-5',
 'wind-uwnd-250-2010-15',
 'wind-uwnd-250-2010-4',
 'nmme0-prate-56w__nasa0',
 'nmme0-prate-34w__c

In [42]:
X = train_df[features].values
y = train_df[target[0]].values

skf = KFold(n_splits=5, random_state=2048, shuffle=True)
skf.get_n_splits(X, y)

train_index_list = []
test_index_list = []
train_x_list = []
val_x_list = []
train_y_list = []
val_y_list = []

for i, (train_index, test_index) in tqdm(enumerate(skf.split(X, y))):
    train_index_list.append(train_index)
    test_index_list.append(test_index)

    train_x_list.append(X[train_index, :])
    val_x_list.append(X[test_index, :])
    train_y_list.append(y[train_index])
    val_y_list.append(y[test_index])    
def train_catboost(x, y):
    print(x.shape, y.shape)
    
    cat_models = []
    for i in tqdm(range(5)):
        train_x, val_x = train_x_list[i], val_x_list[i]
        train_y, val_y = train_y_list[i], val_y_list[i]
        clf = CatBoostRegressor(iterations=5000, verbose=200, task_type="GPU",
                           devices='0:1')
        clf.fit(train_x, train_y, eval_set=[(val_x, val_y)], verbose=200, early_stopping_rounds=100)
        cat_models.append(clf)
    
    return cat_models
cat_models = train_catboost(X, y)

preds = []
for cat_model in tqdm(cat_models):
    preds.append(cat_model.predict(test_df[features].values))
s = 0
for pred in preds:
    s += pred
s = s/5
res_df = pd.read_csv('./sample_solution.csv')
res_df['contest-tmp2m-14d__tmp2m'] = s
res_df.to_csv('submission_fe_v1.csv', index=False)

best_df = pd.read_csv('best_sub.csv')
y_best = best_df[target[0]].values

from numpy import dot
from numpy.linalg import norm

cos_sim = dot(y_best, s)/(norm(y_best)*norm(s))
print("cos_sim with best submission:", cos_sim)

5it [00:04,  1.12it/s]


(375734, 68) (375734,)


  0%|                                                                                                | 0/5 [00:00<?, ?it/s]

Learning rate set to 0.072699
0:	learn: 9.2611105	test: 9.2407165	best: 9.2407165 (0)	total: 5.91ms	remaining: 29.5s
200:	learn: 1.1779071	test: 1.1829674	best: 1.1829674 (200)	total: 1.01s	remaining: 24.3s
400:	learn: 0.9256625	test: 0.9344083	best: 0.9344083 (400)	total: 1.95s	remaining: 22.4s
600:	learn: 0.7892594	test: 0.8003668	best: 0.8003668 (600)	total: 2.91s	remaining: 21.3s
800:	learn: 0.7036006	test: 0.7165281	best: 0.7165281 (800)	total: 3.88s	remaining: 20.3s
1000:	learn: 0.6452584	test: 0.6595936	best: 0.6595936 (1000)	total: 4.83s	remaining: 19.3s
1200:	learn: 0.6026241	test: 0.6184139	best: 0.6184139 (1200)	total: 5.9s	remaining: 18.7s
1400:	learn: 0.5674995	test: 0.5843671	best: 0.5843671 (1400)	total: 7.02s	remaining: 18s
1600:	learn: 0.5381448	test: 0.5560928	best: 0.5560928 (1600)	total: 8.19s	remaining: 17.4s
1800:	learn: 0.5122026	test: 0.5311826	best: 0.5311826 (1800)	total: 9.39s	remaining: 16.7s
2000:	learn: 0.4905743	test: 0.5105235	best: 0.5105235 (2000)	tota

 20%|█████████████████▌                                                                      | 1/5 [00:33<02:12, 33.11s/it]

...

Learning rate set to 0.072699
0:	learn: 9.2604847	test: 9.2566518	best: 9.2566518 (0)	total: 5.83ms	remaining: 29.1s
200:	learn: 1.1679634	test: 1.1825275	best: 1.1825275 (200)	total: 1.02s	remaining: 24.5s
400:	learn: 0.9191806	test: 0.9327356	best: 0.9327356 (400)	total: 1.95s	remaining: 22.4s
600:	learn: 0.7854239	test: 0.7994369	best: 0.7994369 (600)	total: 2.96s	remaining: 21.7s
800:	learn: 0.7061838	test: 0.7211518	best: 0.7211518 (800)	total: 3.99s	remaining: 20.9s
1000:	learn: 0.6450321	test: 0.6612958	best: 0.6612958 (1000)	total: 4.94s	remaining: 19.7s
1200:	learn: 0.6033919	test: 0.6208225	best: 0.6208225 (1200)	total: 5.89s	remaining: 18.6s
1400:	learn: 0.5661044	test: 0.5847533	best: 0.5847533 (1400)	total: 6.88s	remaining: 17.7s
1600:	learn: 0.5362042	test: 0.5560059	best: 0.5560059 (1600)	total: 7.83s	remaining: 16.6s
1800:	learn: 0.5112943	test: 0.5317213	best: 0.5317213 (1800)	total: 8.89s	remaining: 15.8s
2000:	learn: 0.4895477	test: 0.5109915	best: 0.5109915 (2000)	t

 80%|██████████████████████████████████████████████████████████████████████▍                 | 4/5 [02:14<00:34, 34.02s/it]

Learning rate set to 0.072699
0:	learn: 9.2625918	test: 9.2370575	best: 9.2370575 (0)	total: 6ms	remaining: 30s
200:	learn: 1.1658249	test: 1.1729349	best: 1.1729349 (200)	total: 1.01s	remaining: 24.2s
400:	learn: 0.9121833	test: 0.9209001	best: 0.9209001 (400)	total: 1.93s	remaining: 22.2s
600:	learn: 0.7808941	test: 0.7916817	best: 0.7916817 (600)	total: 2.9s	remaining: 21.2s
800:	learn: 0.7034341	test: 0.7161577	best: 0.7161577 (800)	total: 3.83s	remaining: 20.1s
1000:	learn: 0.6438365	test: 0.6581102	best: 0.6581102 (1000)	total: 4.77s	remaining: 19.1s
1200:	learn: 0.5990449	test: 0.6143275	best: 0.6143275 (1200)	total: 5.69s	remaining: 18s
1400:	learn: 0.5622990	test: 0.5788888	best: 0.5788888 (1400)	total: 6.68s	remaining: 17.2s
1600:	learn: 0.5327785	test: 0.5503545	best: 0.5503545 (1600)	total: 7.61s	remaining: 16.2s
1800:	learn: 0.5084866	test: 0.5271642	best: 0.5271642 (1800)	total: 8.51s	remaining: 15.1s
2000:	learn: 0.4874190	test: 0.5070817	best: 0.5070817 (2000)	total: 9.

100%|████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 26.34it/s]


cos_sim with best submission: 0.999829699777029
